In [ ]:
!open .

In [ ]:
import pandas as pd
import requests
import bs4
import math
import time
import NeoQuant as nq
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## get all company list

In [ ]:
companies = nq.get_all_company_list()
companies.index = 'A' + companies.index

## make company code list

In [ ]:
company_code_list = companies.index

In [ ]:
company_name_list = ['신세계']
company_code_list = nq.get_company_code_list(company_name_list)

## get fnguide info and make base, fr data_frame

In [ ]:
import time

total = len(company_code_list)

for num, company_code in enumerate(company_code_list):
    try:
        company_name = nq.get_company_name(company_code, companies)
#         print(num, company_code, company_name)
        print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')

        time.sleep(1)

        try:
            snapshot_tables = nq.request_fnguide_snapshot(company_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            snapshot_tables = fs_df = nq.request_fnguide_snapshot(company_code)

        if len(snapshot_tables) < 10:
            print('>>>> not much data')
            continue
        temp_df = nq.make_basic_df(company_code, snapshot_tables)
        temp_df['name'] = company_name
        if num == 0:
            basic_df = temp_df
        else:
            basic_df = pd.concat([basic_df, temp_df], sort=False)

        temp_df = nq.make_fr_df(company_code, snapshot_tables)
        if num == 0:
            fr_df = temp_df
        else:
            fr_df = pd.concat([fr_df, temp_df], sort=False)
    except ValueError:
        print('>>>> ValueError')
        continue
    except TypeError:
        print('>>>> TypeError')
        continue
    except KeyError:
        print('>>>> KeyError')
        continue
    
        
    if num == 20:
        break


## save basic dataframe to excel

In [ ]:
basic_df.to_excel(r'data/basic_data.xlsx')

## load basic dataframe from excel

In [ ]:
basic_df = pd.read_excel(r'data/basic_data.xlsx')
basic_df.index = basic_df[basic_df.columns[0]]
basic_df.index.name = ''
basic_df.drop(basic_df.columns[0], axis = 1, inplace = True)

## save fr dataframe to excel

In [ ]:
fr_df.to_excel(r'data/fr_data.xlsx')

## load fr dataframe from excel

In [ ]:
fr_df = pd.read_excel(r'data/fr_data.xlsx')
fr_df.index = fr_df[fr_df.columns[0]]
fr_df.index.name = ''
fr_df.drop(fr_df.columns[0], axis = 1, inplace = True)
big_col = list(fr_df.columns)
small_col = list(fr_df.iloc[0])

new_big_col = []
for num, col in enumerate(big_col):
    if 'Unnamed' in col:
        new_big_col.append(new_big_col[num-1])
    else:
        new_big_col.append(big_col[num])

fr_df.columns = [new_big_col, small_col]
fr_df = fr_df.loc[ fr_df.index.dropna() ]